In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import re

In [104]:
df = pd.read_csv('data/training.csv')

y = df['claim_amount']
df = df.drop(columns=['claim_amount'])

def wrangle(df):
    
    original_len = len(df)
    
    # Replace 0 vehicle weight with mean
    df.vh_weight = df.vh_weight.replace(0.0, np.mean(df[df.vh_weight > 0].vh_weight))
    
    # Replace NaNs with column mean
    nans = ['vh_age', 'vh_speed', 'vh_value', 'vh_weight']
    df[nans] = df[nans].fillna(df[nans].mean())
    
    # Join first year data
    df = df.merge(df[df.year == 1.0][['id_policy', 'pol_no_claims_discount']],
                  on='id_policy', suffixes=('', '_first'), how='left')
    
    # Change from beginning discount level
    df['discount_base_change'] = df.pol_no_claims_discount - 0.631
    # Yearly discount change over licence ownership
    df['discount_yearly_change'] = df.discount_base_change / df.drv_age_lic1
    
    # Discount change from policy beginning
    df['discount_change'] = df.pol_no_claims_discount - df.pol_no_claims_discount_first
    # Approx. no. of claims since first year
    df['no_claims'] = np.maximum(np.zeros_like(df.year), np.ceil(df.discount_change / 0.2))
    
    # Driver 1 and 2 combined info
    df['drv_sex2'] = df.drv_sex2.replace('0', '')
    df['drv_sexes'] = df.apply(lambda row: ''.join(sorted(row.drv_sex1 + row.drv_sex2)), axis=1)
    df['drv_avg_age'] = np.mean(df[['drv_age1', 'drv_age2']], axis=1)
    df['drv_avg_lic'] = np.mean(df[['drv_age_lic1', 'drv_age_lic2']], axis=1)
    
    # Population density
    df['pop_dens'] = df.population / df.town_surface_area
    
    # Drop unnecessary cols
    df = df.drop(columns=['id_policy', 'drv_drv2', 'drv_sex2', 'drv_age2', 'drv_age_lic2',
                         'vh_make_model', 'pol_pay_freq', 'pol_no_claims_discount_first'])
    
    # One-hot encoding for categorical variables
    cats = ['pol_coverage', 'pol_payd', 'pol_usage', 'drv_sex1', 'vh_fuel', 'vh_type',
           'drv_sexes']
    df = pd.get_dummies(df, prefix=cats,
                       columns=cats)
    
    assert len(df) == original_len
    return df

def normalize(df, normalizer=None):
    
    cats = ['pol_coverage', 'pol_pay_freq', 'pol_payd', 'pol_usage', 'drv_sex1', 'vh_fuel', 'vh_type',
           'drv_sexes']
    dont_normalize = ['pol_no_claims_discount', 'discount_base_change', 'discount_yearly_change',
                                                                     'discount_change']
    
    # Don't normalize categorical variables nor those in dont_normalize
    to_normalize = [elem for elem in list(df.columns) if elem not in dont_normalize 
                    and '_'.join(elem.split('_')[:-1]) not in cats]

    if normalizer is None:
        normalizer = StandardScaler()
        normalizer = normalizer.fit(df[to_normalize])
        
    df[to_normalize] = normalizer.transform(df[to_normalize])
    return df, normalizer

df = wrangle(df)

y = y[df.year != 1]
df = df[df.year != 1]

df = df.drop(columns=['year'])

x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=2021)
# x_train = df[df.year!=4]
# y_train = y[df.year!=4]
# x_test = df[df.year==4]
# y_test = y[df.year==4]

# x_train, norm = normalize(x_train)
# x_test, blah = normalize(x_test, norm)

In [105]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)

train_preds = reg.predict(x_train)
train_preds[train_preds < 0] = 0

lr_test_preds = reg.predict(x_test)
lr_test_preds[lr_test_preds < 0] = 0

train_error = mean_squared_error(y_train, train_preds, squared=False)
test_error = mean_squared_error(y_test, lr_test_preds, squared=False)

print("Train: {}\nTest: {}".format(train_error, test_error))

Train: 696.3502282964719
Test: 664.7405171664835


In [106]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.4, 0.5, 0.8, 1, 1.5]
ridge = RidgeCV(alphas=alphas)
ridge.fit(x_train, y_train)

train_preds = ridge.predict(x_train)
train_preds[train_preds < 0] = 0

test_preds = ridge.predict(x_test)
test_preds[test_preds < 0] = 0

train_error = mean_squared_error(y_train, train_preds, squared=False)
test_error = mean_squared_error(y_test, test_preds, squared=False)

print("Train: {}\nTest: {}".format(train_error, test_error))

Train: 696.3540169594413
Test: 664.7430345538231


In [107]:
from sklearn.ensemble import RandomForestRegressor

n_estimators = 150
max_depth = 12
max_features = 'log2'
min_samples_split = 200
max_leaf_nodes = None

reg = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_leaf_nodes=max_leaf_nodes,
                             max_features=max_features, min_samples_split=min_samples_split, bootstrap=True).fit(x_train, y_train)

train_preds = reg.predict(x_train)
train_preds[train_preds < 0] = 0

rf_test_preds = reg.predict(x_test)
rf_test_preds[rf_test_preds < 0] = 0

train_error = mean_squared_error(y_train, train_preds, squared=False)
test_error = mean_squared_error(y_test, rf_test_preds, squared=False)

print("Train: {}\nTest: {}".format(train_error, test_error))

Train: 683.51767538742
Test: 664.6873866470963


In [108]:
for name, importance in zip(x_train.columns, reg.feature_importances_):
     print(name, "=", importance)

pol_no_claims_discount = 0.01751669299071512
pol_duration = 0.02956283227049652
pol_sit_duration = 0.03452078976389694
drv_age1 = 0.037904704401577935
drv_age_lic1 = 0.032696496223530463
vh_age = 0.08130827072239286
vh_speed = 0.04114292158388764
vh_value = 0.06898941608568825
vh_weight = 0.06536773022503865
population = 0.05646366626188725
town_surface_area = 0.061219250091820415
discount_base_change = 0.02007273858490973
discount_yearly_change = 0.04094374893338156
discount_change = 0.034299530145711236
no_claims = 0.007732737993223133
drv_avg_age = 0.04031541041432674
drv_avg_lic = 0.03282895913590297
pop_dens = 0.11466975185601756
pol_coverage_Max = 0.07327066923034933
pol_coverage_Med1 = 0.007312342953147885
pol_coverage_Med2 = 0.010737258036903917
pol_coverage_Min = 0.012184015752246717
pol_payd_No = 0.0013261091750704128
pol_payd_Yes = 0.002118938704103805
pol_usage_AllTrips = 0.00611343795416668
pol_usage_Professional = 0.009435697097646873
pol_usage_Retired = 0.003070845644325

In [109]:
import xgboost as xgb

reg = xgb.XGBRegressor(
    n_estimators=15,
    reg_lambda=0.001,
    gamma=1,
    max_depth=3
)

reg.fit(x_train, y_train)

xg_train_preds = reg.predict(x_train)
xg_train_preds[xg_train_preds < 0] = 0

xg_test_preds = reg.predict(x_test)
xg_test_preds[xg_test_preds < 0] = 0

train_error = mean_squared_error(y_train, xg_train_preds, squared=False)
test_error = mean_squared_error(y_test, xg_test_preds, squared=False)

print("Train: {}\nTest: {}".format(train_error, test_error))

Train: 693.4142047405898
Test: 664.913363236186


In [110]:
for name, importance in zip(df.columns, reg.feature_importances_):
     print(name, "=", importance)

pol_no_claims_discount = 0.02393794
pol_duration = 0.011678419
pol_sit_duration = 0.23209405
drv_age1 = 0.012909182
drv_age_lic1 = 0.0
vh_age = 0.044964578
vh_speed = 0.020832509
vh_value = 0.037935458
vh_weight = 0.015009883
population = 0.048445407
town_surface_area = 0.018680349
discount_base_change = 0.0
discount_yearly_change = 0.011333882
discount_change = 0.0354108
no_claims = 0.0
drv_avg_age = 0.007326959
drv_avg_lic = 0.0080668675
pop_dens = 0.025187917
pol_coverage_Max = 0.35941625
pol_coverage_Med1 = 0.0
pol_coverage_Med2 = 0.0
pol_coverage_Min = 0.0055259084
pol_payd_No = 0.0
pol_payd_Yes = 0.0
pol_usage_AllTrips = 0.02709083
pol_usage_Professional = 0.021446051
pol_usage_Retired = 0.0
pol_usage_WorkPrivate = 0.0
drv_sex1_F = 0.0
drv_sex1_M = 0.0
vh_fuel_Diesel = 0.014914292
vh_fuel_Gasoline = 0.005794684
vh_fuel_Hybrid = 0.0
vh_type_Commercial = 0.0
vh_type_Tourism = 0.0
drv_sexes_F = 0.011997804
drv_sexes_FF = 0.0
drv_sexes_FM = 0.0
drv_sexes_M = 0.0
drv_sexes_MM = 0.0


In [111]:
a = np.mean([rf_test_preds, test_preds], axis=0)
mean_squared_error(y_test, a, squared=False)

664.5209808977036

In [112]:
b = np.minimum(rf_test_preds, test_preds)
mean_squared_error(y_test, b, squared=False)

664.5645457297345

In [113]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(n_estimators=65, learning_rate=0.5,
     max_depth=1, random_state=2021, loss='ls', criterion='mse').fit(x_train, y_train)

train_preds = reg.predict(x_train)
train_preds[train_preds < 0] = 0

test_preds = reg.predict(x_test)
test_preds[test_preds < 0] = 0

train_error = mean_squared_error(y_train, train_preds, squared=False)
test_error = mean_squared_error(y_test, test_preds, squared=False)

print("Train: {}\nTest: {}".format(train_error, test_error))

Train: 695.9765715478296
Test: 664.6000317820037
